In [1]:
%pip install -r requirements.txt

  Cloning https://github.com/lvwerra/trl to /tmp/pip-req-build-6za_h6z4
  Running command git clone --filter=blob:none --quiet https://github.com/lvwerra/trl /tmp/pip-req-build-6za_h6z4
  Resolved https://github.com/lvwerra/trl to commit 5fb5af7c3410de3e21edc4479f646b64aee2ff20
  Preparing metadata (setup.py) ... done
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-gy8mdhgv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-gy8mdhgv
  Resolved https://github.com/huggingface/transformers.git to commit 17a55534f5e5df10ac4804d4270bf6b8cc24998d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the

In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from accelerate import Accelerator
from datasets import load_dataset
from peft import LoraConfig
from tqdm import tqdm
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, TrainingArguments, logging, set_seed

from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset


/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/jhoffbauer/reddit_qa/venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [5]:
for model_name in ['EleutherAI/pythia-70M', 'EleutherAI/pythia-160M', 'EleutherAI/pythia-1B', 'EleutherAI/pythia-2.8B']:
    for load_in_8bit in [False, True]:    
        if load_in_8bit:
            model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map={"": Accelerator().process_index})
        else: 
            model = AutoModelForCausalLM.from_pretrained(model_name)
        
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        prompt = "Q: Who is the current president of the United States? \nA: "
        inputs = tokenizer(prompt, return_tensors="pt")
        outputs = model.generate(**inputs, max_new_tokens=20, num_beams=10)
        output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        print(model_name + (' /8bit' if load_in_8bit else ''))
        print(f'{output[0]}')
        print('-' * 8)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


EleutherAI/pythia-70M
Q: Who is the current president of the United States? 
A: 
The current president of the United States is the current president of the United States.  He is
--------


RuntimeError: No CUDA GPUs are available

In [8]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

def prepare_sample_text(example):
    """Prepare the text from a sample of the dataset."""
    text = f"Question: {example['question']}\n\nAnswer: {example['response_j']}"
    return text

def create_datasets(tokenizer, args):
    dataset = load_dataset(
        args.dataset_name,
        data_dir=args.subset,
        split=args.split,
        use_auth_token=True,
        num_proc=args.num_workers if not args.streaming else None,
        streaming=args.streaming,
    )
    if args.streaming:
        print("Loading the dataset in streaming mode")
        valid_data = dataset.take(args.size_valid_set)
        train_data = dataset.skip(args.size_valid_set)
        train_data = train_data.shuffle(buffer_size=args.shuffle_buffer, seed=args.seed)
    else:
        dataset = dataset.train_test_split(test_size=0.005, seed=args.seed)
        train_data = dataset["train"]
        valid_data = dataset["test"]
        print(f"Size of the train set: {len(train_data)}. Size of the validation set: {len(valid_data)}")

    chars_per_token = chars_token_ratio(train_data, tokenizer)
    print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

    train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        formatting_func=prepare_sample_text,
        infinite=True,
        seq_length=args.seq_length,
        chars_per_token=chars_per_token,
    )
    valid_dataset = ConstantLengthDataset(
        tokenizer,
        valid_data,
        formatting_func=prepare_sample_text,
        infinite=False,
        seq_length=args.seq_length,
        chars_per_token=chars_per_token,
    )
    return train_dataset, valid_dataset

In [7]:
max_steps = 10000
batch_size = 4
learning_rate=1e-4
run_name = "test_run"
model_path = "EleutherAI/pythia-70M"


lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

print("Starting main loop")

training_args = TrainingArguments(
    output_dir=f'/scratch1/jhoff/runs/{run_name}',
    dataloader_drop_last=True,
    evaluation_strategy="steps",
    max_steps=max_steps,
    eval_steps=1000,
    save_steps=1000,
    logging_steps=1000,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,
    fp16=False,
    bf16=False,
    weight_decay=0.05,
    run_name=run_name,
    report_to="wandb",
    ddp_find_unused_parameters=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_path
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    peft_config=lora_config,
    packing=True,
)

print_trainable_parameters(trainer.model)

print("Training...")
trainer.train()

print("Saving last checkpoint of the model")
trainer.model.save_pretrained(os.path.join(args.output_dir, "final_checkpoint/"))


In [ ]:
config = AutoConfig.from_pretrained(args.model_path)
architecture = config.architectures[0]

tokenizer = AutoTokenizer.from_pretrained(args.model_path)

if "Llama" in architecture:
    print("Setting EOS, BOS, and UNK tokens for LLama tokenizer")
    tokenizer.add_special_tokens(
        {
            "eos_token": "</s>",
            "bos_token": "</s>",
            "unk_token": "</s>",
        }
    )

train_dataset, eval_dataset = create_datasets(tokenizer, args)
run_training(args, train_dataset, eval_dataset)